## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.9180,25.5701,66.49,96,0,9.22,clear sky
1,1,Kijang,KR,35.2442,129.2139,44.33,46,97,11.39,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,66.22,68,0,6.91,light rain
3,3,Bluff,NZ,-46.6000,168.3333,58.53,75,10,1.95,clear sky
4,4,Tukrah,LY,32.5341,20.5791,63.48,36,99,12.06,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Atuona,PF,-9.8000,-139.0333,78.91,79,60,12.66,broken clouds
14,14,Butaritari,KI,3.0707,172.7902,81.03,65,90,7.67,overcast clouds
18,18,Rikitea,PF,-23.1203,-134.9692,79.14,66,14,9.44,few clouds
19,19,Hithadhoo,MV,-0.6000,73.0833,81.63,65,6,6.17,clear sky
20,20,Berbera,SO,10.4396,45.0143,79.61,64,11,5.35,few clouds
26,26,Kiunga,PG,-6.1219,141.2906,78.57,88,77,2.57,broken clouds
29,29,Chirongui,YT,-12.9319,45.1483,76.93,76,79,0.74,broken clouds
36,36,Kapaa,US,22.0752,-159.3190,79.48,65,20,8.05,few clouds
49,49,San Patricio,US,28.0170,-97.5169,85.53,44,99,17.58,overcast clouds
51,51,Dingle,PH,10.9995,122.6711,78.44,84,73,7.83,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                563
City                   563
Country                562
Lat                    563
Lng                    563
Max Temp               563
Humidity               563
Cloudiness             563
Wind Speed             563
Current Description    563
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities=preferred_cities_df.dropna()
pref_cities.count()

City_ID                562
City                   562
Country                562
Lat                    562
Lng                    562
Max Temp               562
Humidity               562
Cloudiness             562
Wind Speed             562
Current Description    562
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Atuona,PF,78.91,broken clouds,-9.8000,-139.0333,
14,Butaritari,KI,81.03,overcast clouds,3.0707,172.7902,
18,Rikitea,PF,79.14,few clouds,-23.1203,-134.9692,
19,Hithadhoo,MV,81.63,clear sky,-0.6000,73.0833,
20,Berbera,SO,79.61,few clouds,10.4396,45.0143,
26,Kiunga,PG,78.57,broken clouds,-6.1219,141.2906,
29,Chirongui,YT,76.93,broken clouds,-12.9319,45.1483,
36,Kapaa,US,79.48,few clouds,22.0752,-159.3190,
49,San Patricio,US,85.53,overcast clouds,28.0170,-97.5169,
51,Dingle,PH,78.44,broken clouds,10.9995,122.6711,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [21]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Atuona,PF,78.91,broken clouds,-9.8000,-139.0333,Villa Enata
14,Butaritari,KI,81.03,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
18,Rikitea,PF,79.14,few clouds,-23.1203,-134.9692,People ThankYou
19,Hithadhoo,MV,81.63,clear sky,-0.6000,73.0833,Scoop Guest House
20,Berbera,SO,79.61,few clouds,10.4396,45.0143,IBCO REAL ESTATE PROPERTY


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   562
Country                562
Max Temp               562
Current Description    562
Lat                    562
Lng                    562
Hotel Name             562
dtype: int64

In [25]:
# 8a. Create the output File (CSV)
output_data_file= "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))